# <font size=20>**Domino Experiment Metadata Summary**</font>

<font color = "\#8FBC8F"> Cells with this color must run for code to function</font>

---

# <font color = "\#8FBC8F"> Init


In [ ]:
#@title <font color = "\#8FBC8F"> Google Drive mount
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#@title <font color = "\#8FBC8F">Imports

import os
import numpy as np
import matplotlib.pyplot as plt

!pip install mne --upgrade
import mne
print('[imports successfully loaded]')

     |████████████████████████████████| 6.9MB 5.9MB/s 
[imports successfully loaded]


In [ ]:
#@title Subject Class

class Subject:
    def __init__(self, name, data_dir=None, pf=False):
        """
        :param name: patient name - 'D0**' is the general format
        :param data_dir: str - path to the directory containing the .set files
        """
        self.data_dir = data_dir
        self.name = name
        self.set_names = ['match', 'non_match']

        # labels are set such that a 'dangerous' choice is 1 and a 'safe' choice is 0
        self.labels = {self.set_names[0]: 0, self.set_names[1]: 1}
        self.sets = {}
        if self.data_dir: self.load_sets(self.data_dir)
        if pf: print(self)

    def __getitem__(self, item):
        """
        :param item: item number
        :return: mne.Epochs. set
        """
        name = self.set_names[item]
        print(f'Set Name: {name}, label: {self.labels[name]}')
        return self.sets[self.set_names[item]]

    def __repr__(self):
        return 'Subject\'s sets by number: (can refer to as Patient[number])\n' + str(self)

    def __str__(self):
        return ''.join([f'[{i}]: {name}\n' for i, name in enumerate(self.set_names)])

    def load_sets(self, data_dir):
        """
        loads the sets' data to mne.Epoch objects
        :param data_dir: str - path to the directory containing the .set files
        """
        path = f"{data_dir}{self.name}/eeglab/"
        files = [f for f in os.listdir(path) if 'Picked' in f]
        for f in files:
            self.sets[f[12:-9]] = mne.read_epochs_eeglab(path + f)

    def _get_set_name(self, set_identifier=None):
        """
        allows for user live set name choice via name/number
        :return: the name of the wanted set
        """
        try:
            if isinstance(set_identifier, int):
                return self.set_names[set_identifier]
            elif isinstance(set_identifier, str):
                return set_identifier
            else:
                return self.set_names[int(input(f'what Epochs element do you want to edit?\n' + str(self)))]
        except Exception as e:
            raise e

    #TODO: Fix this - both sets hav match in their name
    def reset_set(self, set_id=None):
        """
        Reloads the specified set
        :param set_id: set to reload via name/number
        """
        set_name = self._get_set_name(set_id)
        print(f'[Resetting set:{set_name}]')
        path = f'{self.data_dir}{self.name}\\eeglab\\'
        fl = [f for f in os.listdir(path) if set_name in f]
        print(f'[set file: {path + fl[0]}')
        self.sets[set_name] = mne.io.read_epochs_eeglab(path + file[0])

    def plot_set(self, set_id=None):
        """
        opens a mne.Epoch.plot() instance - allows for visual channel and/or epoch drop
        :param set_id: set to reload via name/number
        """
        set_name = self._get_set_name(set_id)
        print(f'[Editing set: {set_name}]')
        self.sets[set_name].plot(scalings={'eeg': 1e-3})

    def plot_set_butterfly(self, set_id=None):
        """
        opens a mne.Epoch.plot() instance - allows for visual channel and/or epoch drop. this opens in butterfly mode
        :param set_id: set to reload via name/number
        """
        set_name = self._get_set_name(set_id)
        print(f'[Editing set: {set_name}]')
        self.sets[set_name].plot(scalings={'eeg': 2e-4}, butterfly=True)

    def edit_set(self, set_id=None):
        """ alias for plot_set """
        self.plot_set(set_id)

    def filter_set(self, w_c=85.0, pf=False, set_id=None):
        """
        filters a set using a LPF with cutoff frequency w_c
        :param w_c: float. cuttof frequency for LPF (default=85Hz)
        :param pf: bool. plot flag - if True, PSD is plotted before and after filtering
        :param set_id: set to reload via name/number
        """
        set_name = self._get_set_name(set_id)  # get set number
        if pf: self.sets[set_name].plot_psd()
        self.sets[set_name].filter(l_freq=124.0, h_freq=w_c)
        if pf: self.sets[set_name].plot_psd()

    def filter_sets(self, w_c=85.0, pf=False):
        """
        filters the data using a LPF with cutoff frequency w_c
        :param w_c: float. cuttof frequency for LPF (default=85Hz)
        :param pf: bool. plot flag - if True, PSD is plotted before and after filtering
        """
        for name in self.set_names:
            self.filter_set(w_c, pf, name)

    def notch_set(self, freqs=None, pf=False, set_id=None):
        """
        filters the set using a notch filter
        :param freqs: frequencies to notch at, can be a float or list of floats(or integers)
        :param pf: bool. plot flag - if True, PSD is plotted before and after filtering
        :param set_id: set to reload via name/number
        """
        if freqs is None: freqs = [float(input('Notch frequency: '))]
        if not isinstance(freqs, np.ndarray): freqs = np.array(freqs)  # turn into numpy array

        set_name = self._get_set_name(set_id)  # get set number
        if pf: self.sets[set_name].plot_psd()
        
        self.sets[set_name]._data = notch(self.sets[set_name].get_data(), 250, freqs=freqs)
        if pf: self.sets[set_name].plot_psd()

    def notch_sets(self, freqs=None, pf=False):
        """
        filters the data using a notch filter
        :param freqs: frequencies to notch at, can be a float or list of floats(or integers)
        :param pf: bool. plot flag - if True, PSD is plotted before and after filtering
        """
        if freqs is None: freqs = [float(input('Notch frequency: '))]
        for name in self.set_names:
            self.notch_set(freqs, pf, name)

    # methods from here on will need revision @@@@
    def get_data_and_label(self):
        if not self.sets: raise PermissionError('Can\'t retrieve data before Epochs are loaded')

        X = self.sets[self.set_names[0]].get_data()
        y = self.labels[self.set_names[0]] * np.ones(X.shape[0])
        for rec_set in self.set_names[1:-1]:
            dt = self.sets[rec_set].get_data()
            X = np.concatenate((X, dt))
            y = np.concatenate((y, self.labels[rec_set] * np.ones(dt.shape[0])))
        return X, y

    def get_CSP(self, X=None, y=None):
        if X is None and y is None:
            X, y = self.get_data_and_label()
        X_train, X_test, y_train, y_test = self.split_data(X, y)
        return CSP().fit(X_train, y_train)
    # till here @@@@

    @staticmethod
    def split_data(X, y):
        return Split(X, y, test_size=0.20)

    def get_morlet(self, n, freqs, n_cycles=3):
        print('Calculating Morlet Wavelet Transform')
        return morl(self[n], freqs=freqs, n_cycles=n_cycles, average=False, return_itc=False)

print('[Class Subject successfully loaded]')


[Class Subject successfully loaded]



---

# <font color = "\#8FBC8F">**Data loading**</font> <font size = 4>(might take some time)</font>

In [ ]:
#@title <font color = "\#8FBC8F">Data loading</font>
DATA_DIR = r'/content/drive/My Drive/Colab Notebooks/Project Domino/Macros/'

names = os.listdir(DATA_DIR)
# print(names)

subj_channels = {}
for name in names:
    try:
        subj = Subject(name=name, data_dir=DATA_DIR)
        subj_channels[name] = subj[0].ch_names
    except Exception as e:
        print("#"*80)
        print(name)
        print("#"*80)
        raise e

print('[Data successfully loaded]')

Extracting parameters from /content/drive/My Drive/Colab Notebooks/Project Domino/Macros/D016/eeglab/D016_Picked_match_chip.set...
Not setting metadata
Not setting metadata
35 matching events found
No baseline correction applied
0 projection items activated
Ready.


<ipython-input-7-b771afc69a6a>:42: RuntimeWarning: 54 channel names are too long, have been truncated to 15 characters:
['EEG_LAF3_EEG_LAF2', 'EEG_LAF4_EEG_LAF3', 'EEG_LAF5_EEG_LAF4', 'EEG_LAF6_EEG_LAF5', 'EEG_LAF7_EEG_LAF6', 'EEG_LAF8_EEG_LAF7', 'EEG_LdAC2_EEG_LdAC1', 'EEG_LdAC3_EEG_LdAC2', 'EEG_LdAC4_EEG_LdAC3', 'EEG_LdAC5_EEG_LdAC4', 'EEG_LdAC6_EEG_LdAC5', 'EEG_LdAC7_EEG_LdAC6', 'EEG_LdAC8_EEG_LdAC7', 'EEG_LOFAC2_EEG_LOFAC1', 'EEG_LOFAC3_EEG_LOFAC2', 'EEG_LOFAC4_EEG_LOFAC3', 'EEG_LOFAC5_EEG_LOFAC4', 'EEG_LOFAC6_EEG_LOFAC5', 'EEG_LOFAC7_EEG_LOFAC6', 'EEG_LOFAC8_EEG_LOFAC7', 'EEG_LSF2_EEG_LSF1', 'EEG_LSF3_EEG_LSF2', 'EEG_LSF4_EEG_LSF3', 'EEG_LSF5_EEG_LSF4', 'EEG_LSF6_EEG_LSF5', 'EEG_LpCING3_EEG_LpCING2', 'EEG_LpCING4_EEG_LpCING3', 'EEG_LpCING5_EEG_LpCING4', 'EEG_LpCING6_EEG_LpCING5', 'EEG_LpCING7_EEG_LpCING6', 'EEG_LpCING8_EEG_LpCING7', 'EEG_LMH2_EEG_LMH1', 'EEG_LMH3_EEG_LMH2', 'EEG_LMH4_EEG_LMH3', 'EEG_LMH5_EEG_LMH4', 'EEG_LMH6_EEG_LMH5', 'EEG_LMH7_EEG_LMH6', 'EEG_LMH8_EEG_LMH7', 'EE

Extracting parameters from /content/drive/My Drive/Colab Notebooks/Project Domino/Macros/D016/eeglab/D016_Picked_non_match_chip.set...
Not setting metadata
Not setting metadata
51 matching events found
No baseline correction applied
0 projection items activated
Ready.
Set Name: match, label: 0


<ipython-input-7-b771afc69a6a>:42: RuntimeWarning: 54 channel names are too long, have been truncated to 15 characters:
['EEG_LAF3_EEG_LAF2', 'EEG_LAF4_EEG_LAF3', 'EEG_LAF5_EEG_LAF4', 'EEG_LAF6_EEG_LAF5', 'EEG_LAF7_EEG_LAF6', 'EEG_LAF8_EEG_LAF7', 'EEG_LdAC2_EEG_LdAC1', 'EEG_LdAC3_EEG_LdAC2', 'EEG_LdAC4_EEG_LdAC3', 'EEG_LdAC5_EEG_LdAC4', 'EEG_LdAC6_EEG_LdAC5', 'EEG_LdAC7_EEG_LdAC6', 'EEG_LdAC8_EEG_LdAC7', 'EEG_LOFAC2_EEG_LOFAC1', 'EEG_LOFAC3_EEG_LOFAC2', 'EEG_LOFAC4_EEG_LOFAC3', 'EEG_LOFAC5_EEG_LOFAC4', 'EEG_LOFAC6_EEG_LOFAC5', 'EEG_LOFAC7_EEG_LOFAC6', 'EEG_LOFAC8_EEG_LOFAC7', 'EEG_LSF2_EEG_LSF1', 'EEG_LSF3_EEG_LSF2', 'EEG_LSF4_EEG_LSF3', 'EEG_LSF5_EEG_LSF4', 'EEG_LSF6_EEG_LSF5', 'EEG_LpCING3_EEG_LpCING2', 'EEG_LpCING4_EEG_LpCING3', 'EEG_LpCING5_EEG_LpCING4', 'EEG_LpCING6_EEG_LpCING5', 'EEG_LpCING7_EEG_LpCING6', 'EEG_LpCING8_EEG_LpCING7', 'EEG_LMH2_EEG_LMH1', 'EEG_LMH3_EEG_LMH2', 'EEG_LMH4_EEG_LMH3', 'EEG_LMH5_EEG_LMH4', 'EEG_LMH6_EEG_LMH5', 'EEG_LMH7_EEG_LMH6', 'EEG_LMH8_EEG_LMH7', 'EE

Extracting parameters from /content/drive/My Drive/Colab Notebooks/Project Domino/Macros/D015/eeglab/D015_Picked_match_chip.set...
Not setting metadata
Not setting metadata
43 matching events found
No baseline correction applied
0 projection items activated
Ready.


...

<ipython-input-7-b771afc69a6a>:42: RuntimeWarning: 40 channel names are too long, have been truncated to 15 characters:
['EEG_RMH2_EEG_RMH1', 'EEG_RMH3_EEG_RMH2', 'EEG_RMH4_EEG_RMH3', 'EEG_RMH5_EEG_RMH4', 'EEG_RMH6_EEG_RMH5', 'EEG_RMH7_EEG_RMH6', 'EEG_RMH8_EEG_RMH7', 'EEG_RMTG2_EEG_RMTG1', 'EEG_RMTG3_EEG_RMTG2', 'EEG_RMTG4_EEG_RMTG3', 'EEG_RMTG5_EEG_RMTG4', 'EEG_RMTG6_EEG_RMTG5', 'EEG_RMTG7_EEG_RMTG6', 'EEG_RMTG8_EEG_RMTG7', 'EEG_RTP2_EEG_RTP1', 'EEG_RTP3_EEG_RTP2', 'EEG_RTP4_EEG_RTP3', 'EEG_RTP5_EEG_RTP4', 'EEG_RTP6_EEG_RTP5', 'EEG_RTP7_EEG_RTP6', 'EEG_RTP8_EEG_RTP7', 'EEG_RAC2_EEG_RAC1', 'EEG_RAC5_EEG_RAC4', 'EEG_RAC6_EEG_RAC5', 'EEG_RAC7_EEG_RAC6', 'EEG_RAC8_EEG_RAC7', 'EEG_LMH2_EEG_LMH1', 'EEG_LMH3_EEG_LMH2', 'EEG_LMH4_EEG_LMH3', 'EEG_LMH5_EEG_LMH4', 'EEG_LMH6_EEG_LMH5', 'EEG_LMH7_EEG_LMH6', 'EEG_LMH8_EEG_LMH7', 'EEG_LAC2_EEG_LAC1', 'EEG_LAC3_EEG_LAC2', 'EEG_LAC4_EEG_LAC3', 'EEG_LAC5_EEG_LAC4', 'EEG_LAC6_EEG_LAC5', 'EEG_LAC7_EEG_LAC6', 'EEG_LAC8_EEG_LAC7']
  self.sets[f[12:-9]] = m

Extracting parameters from /content/drive/My Drive/Colab Notebooks/Project Domino/Macros/D030/eeglab/D030_Picked_match_chip.set...
Not setting metadata
Not setting metadata
42 matching events found
No baseline correction applied
0 projection items activated
Ready.


<ipython-input-7-b771afc69a6a>:42: RuntimeWarning: 48 channel names are too long, have been truncated to 15 characters:
['EEG_RSF4_EEG_RSF3', 'EEG_RSF5_EEG_RSF4', 'EEG_RSF6_EEG_RSF5', 'EEG_RSF7_EEG_RSF6', 'EEG_RSF8_EEG_RSF7', 'EEG_RPreMotor2_EEG_RPreMotor1', 'EEG_RPreMotor3_EEG_RPreMotor2', 'EEG_RPreMotor4_EEG_RPreMotor3', 'EEG_RPreMotor5_EEG_RPreMotor4', 'EEG_RPreMotor6_EEG_RPreMotor5', 'EEG_RSMA2_EEG_RSMA1', 'EEG_RSMA3_EEG_RSMA2', 'EEG_RSMA4_EEG_RSMA3', 'EEG_RSMA5_EEG_RSMA4', 'EEG_RSMA6_EEG_RSMA5', 'EEG_RSMA7_EEG_RSMA6', 'EEG_RSMA8_EEG_RSMA7', 'EEG_ROF2_EEG_ROF1', 'EEG_ROF3_EEG_ROF2', 'EEG_ROF4_EEG_ROF3', 'EEG_ROF5_EEG_ROF4', 'EEG_ROF6_EEG_ROF5', 'EEG_ROF7_EEG_ROF6', 'EEG_ROF8_EEG_ROF7', 'EEG_RAC2_EEG_RAC1', 'EEG_RAC3_EEG_RAC2', 'EEG_RAC4_EEG_RAC3', 'EEG_RAC5_EEG_RAC4', 'EEG_RAC6_EEG_RAC5', 'EEG_RdACC2_EEG_RdACC1', 'EEG_RdACC3_EEG_RdACC2', 'EEG_RdACC4_EEG_RdACC3', 'EEG_RdACC5_EEG_RdACC4', 'EEG_RdACC6_EEG_RdACC5', 'EEG_RdACC7_EEG_RdACC6', 'EEG_RdACC8_EEG_RdACC7', 'EEG_RMidCing2_EEG_RM

Extracting parameters from /content/drive/My Drive/Colab Notebooks/Project Domino/Macros/D030/eeglab/D030_Picked_non_match_chip.set...
Not setting metadata
Not setting metadata
46 matching events found
No baseline correction applied
0 projection items activated
Ready.
Set Name: match, label: 0
[Data successfully loaded]


<ipython-input-7-b771afc69a6a>:42: RuntimeWarning: 48 channel names are too long, have been truncated to 15 characters:
['EEG_RSF4_EEG_RSF3', 'EEG_RSF5_EEG_RSF4', 'EEG_RSF6_EEG_RSF5', 'EEG_RSF7_EEG_RSF6', 'EEG_RSF8_EEG_RSF7', 'EEG_RPreMotor2_EEG_RPreMotor1', 'EEG_RPreMotor3_EEG_RPreMotor2', 'EEG_RPreMotor4_EEG_RPreMotor3', 'EEG_RPreMotor5_EEG_RPreMotor4', 'EEG_RPreMotor6_EEG_RPreMotor5', 'EEG_RSMA2_EEG_RSMA1', 'EEG_RSMA3_EEG_RSMA2', 'EEG_RSMA4_EEG_RSMA3', 'EEG_RSMA5_EEG_RSMA4', 'EEG_RSMA6_EEG_RSMA5', 'EEG_RSMA7_EEG_RSMA6', 'EEG_RSMA8_EEG_RSMA7', 'EEG_ROF2_EEG_ROF1', 'EEG_ROF3_EEG_ROF2', 'EEG_ROF4_EEG_ROF3', 'EEG_ROF5_EEG_ROF4', 'EEG_ROF6_EEG_ROF5', 'EEG_ROF7_EEG_ROF6', 'EEG_ROF8_EEG_ROF7', 'EEG_RAC2_EEG_RAC1', 'EEG_RAC3_EEG_RAC2', 'EEG_RAC4_EEG_RAC3', 'EEG_RAC5_EEG_RAC4', 'EEG_RAC6_EEG_RAC5', 'EEG_RdACC2_EEG_RdACC1', 'EEG_RdACC3_EEG_RdACC2', 'EEG_RdACC4_EEG_RdACC3', 'EEG_RdACC5_EEG_RdACC4', 'EEG_RdACC6_EEG_RdACC5', 'EEG_RdACC7_EEG_RdACC6', 'EEG_RdACC8_EEG_RdACC7', 'EEG_RMidCing2_EEG_RM

---

# <font color = "\#8FBC8F">Channel Names</font>

In [ ]:
#@title <font color = "\#8FBCFF">Unipolar</font> channel names per patient
from itertools import groupby
def split_text(ch_name):
    for s in ch_name.split("_"):
        for k, g in groupby(s, str.isalpha):
            yield ''.join(g)

sc = subj_channels.copy()
cn = {}

for k,v in sorted(sc.items()):
    cn[k] = []
    for i, ch_n in enumerate(v):
        clean_name = [n for n in [*split_text(ch_n)][:3] if n != 'EEG']
        # print(clean_name)
        cn[k].append(clean_name)
        v[i] = "_".join(clean_name)
        
    print(f"{k}: {v}")

D006: ['RMH_66', 'RMH_67', 'RMH_68', 'RMH_69', 'RMH_70', 'RMH_71', 'REC_74', 'REC_75', 'REC_76', 'REC_77', 'REC_78', 'REC_79', 'REC_80', 'RA_82', 'RA_83', 'RA_84', 'RA_85', 'RA_86', 'RA_87', 'RA_88', 'RAC_90', 'RAC_91', 'RAC_92', 'RAC_93', 'RAC_94', 'RAC_95', 'RAC_96', 'RMC_98', 'RMC_99', 'RMC_101', 'RMC_102', 'RMC_103', 'RMC_104', 'RCAUD_106', 'RCAUD_107', 'RCAUD_108', 'RCAUD_109', 'RCAUD_110', 'RCAUD_111', 'RCAUD_112', 'RAF_114', 'RAF_115', 'RAF_116', 'RAF_117', 'RAF_118', 'RAF_119', 'RAF_120']
D011: ['RAH_2', 'RAH_3', 'RAH_4', 'RAH_5', 'RAH_6', 'RAH_7', 'RAH_8', 'RaSTG_2', 'RaSTG_2', 'RaSTG_2', 'RaSTG_2', 'RaSTG_2', 'RaSTG_2', 'RaSTG_2', 'RaSTG_1', 'RaSTG_1', 'RaSTG_1', 'RaSTG_1', 'RaSTG_1', 'RaSTG_1', 'RaSTG_1', 'RP_2', 'RP_3', 'RP_4', 'RP_5', 'RP_6', 'RP_7', 'RP_8', 'RpSMA_2', 'RpSMA_3', 'RpSMA_4', 'RpSMA_5', 'RpSMA_6', 'RpSMA_7', 'RpSMA_8', 'RdAC_4', 'RdAC_5', 'RdAC_6', 'RdAC_7', 'RdAC_8', 'RmdAC_2', 'RmdAC_3', 'RmdAC_4', 'RmdAC_5', 'RmdAC_6', 'RmdAC_7', 'RmdAC_8', 'RasOF_2', 'Ra

In [ ]:
#@title <font color = "\#8FBCFF">Bipolar</font> channel names per patient
sc = subj_channels.copy()
for k,v in sorted(sc.items()):
    print(f"{k}: {v}")

D006: ['RMH_66', 'RMH_67', 'RMH_68', 'RMH_69', 'RMH_70', 'RMH_71', 'REC_74', 'REC_75', 'REC_76', 'REC_77', 'REC_78', 'REC_79', 'REC_80', 'RA_82', 'RA_83', 'RA_84', 'RA_85', 'RA_86', 'RA_87', 'RA_88', 'RAC_90', 'RAC_91', 'RAC_92', 'RAC_93', 'RAC_94', 'RAC_95', 'RAC_96', 'RMC_98', 'RMC_99', 'RMC_101', 'RMC_102', 'RMC_103', 'RMC_104', 'RCAUD_106', 'RCAUD_107', 'RCAUD_108', 'RCAUD_109', 'RCAUD_110', 'RCAUD_111', 'RCAUD_112', 'RAF_114', 'RAF_115', 'RAF_116', 'RAF_117', 'RAF_118', 'RAF_119', 'RAF_120']
D011: ['RAH_2', 'RAH_3', 'RAH_4', 'RAH_5', 'RAH_6', 'RAH_7', 'RAH_8', 'RaSTG_2', 'RaSTG_2', 'RaSTG_2', 'RaSTG_2', 'RaSTG_2', 'RaSTG_2', 'RaSTG_2', 'RaSTG_1', 'RaSTG_1', 'RaSTG_1', 'RaSTG_1', 'RaSTG_1', 'RaSTG_1', 'RaSTG_1', 'RP_2', 'RP_3', 'RP_4', 'RP_5', 'RP_6', 'RP_7', 'RP_8', 'RpSMA_2', 'RpSMA_3', 'RpSMA_4', 'RpSMA_5', 'RpSMA_6', 'RpSMA_7', 'RpSMA_8', 'RdAC_4', 'RdAC_5', 'RdAC_6', 'RdAC_7', 'RdAC_8', 'RmdAC_2', 'RmdAC_3', 'RmdAC_4', 'RmdAC_5', 'RmdAC_6', 'RmdAC_7', 'RmdAC_8', 'RasOF_2', 'Ra

In [ ]:
#@title <font color = "\#8FBCFF">Patients' probes 
my_dict = {}
for k,v in sorted(cn.items()):
    MyList = [i[0] for i in cn[k]]
    my_dict[k] = {i:MyList.count(i) for i in MyList}
    print(f"{k}: {my_dict[k]}")

D006: {'RMH': 6, 'REC': 7, 'RA': 7, 'RAC': 7, 'RMC': 6, 'RCAUD': 7, 'RAF': 7}
D011: {'RAH': 7, 'RaSTG': 14, 'RP': 7, 'RpSMA': 7, 'RdAC': 5, 'RmdAC': 7, 'RasOF': 7, 'ROF': 7, 'RaOF': 7, 'RpT': 7}
D013: {'RA': 7, 'RMH': 7, 'RMTG': 7, 'RTP': 7, 'RAC': 5, 'LA': 7, 'LMH': 7, 'LAC': 7}
D014: {'LA': 5, 'LSTG': 7, 'LAF': 5, 'LF': 7, 'LMFG': 7, 'LOF': 7, 'LSMA': 5}
D015: {'RHC': 6, 'ROF': 6, 'ROPR': 6, 'RMF': 7, 'RASF': 7, 'RAC': 5, 'LHC': 7, 'LOF': 7}
D016: {'LAF': 6, 'LdAC': 7, 'LOFAC': 7, 'LSF': 5, 'LpCING': 6, 'LMH': 7, 'LEC': 5, 'RdAC': 7, 'RSF': 4}
D017: {'LOF': 7, 'LAI': 5, 'LMI': 7, 'LSI': 7, 'LPI': 7, 'LSTG': 7, 'LPHG': 7, 'RA': 7, 'REC': 7, 'RMH': 5}
D018: {'RA': 4, 'RMH': 7, 'ROFAC': 6, 'RAC': 7, 'RCAUD': 3, 'RMC': 7, 'RSMA': 7, 'LCAUD': 7, 'LOFAC': 4}
D023: {'RA': 5, 'RMH': 7, 'RPHG': 6, 'RTO': 7, 'ROFAC': 5, 'RF': 7, 'LOF': 5, 'LMH': 7, 'LPHG': 7, 'LO': 5, 'LA': 4}
D030: {'RSF': 5, 'RPreMotor': 5, 'RSMA': 7, 'RF': 4, 'ROF': 7, 'RAC': 5, 'RdACC': 7, 'RMidCing': 5, 'RA': 5, 'RH': 3, 

In [ ]:
#@title <font color = "\#8FBC4F">Probes' patients
ch_per_subj = {}
for key, value in sorted(my_dict.items()):
    for v in value:
        if v not in ch_per_subj.keys(): ch_per_subj[v] = [key]
        else: ch_per_subj[v].append(key)

for k,v in ch_per_subj.items():
    print(f"{k}: {v}")

RMH: ['D006', 'D013', 'D017', 'D018', 'D023']
REC: ['D006', 'D017']
RA: ['D006', 'D013', 'D017', 'D018', 'D023', 'D030']
RAC: ['D006', 'D013', 'D015', 'D018', 'D030']
RMC: ['D006', 'D018']
RCAUD: ['D006', 'D018']
RAF: ['D006']
RAH: ['D011']
RaSTG: ['D011']
RP: ['D011']
RpSMA: ['D011']
RdAC: ['D011', 'D016']
RmdAC: ['D011']
RasOF: ['D011']
ROF: ['D011', 'D015', 'D030']
RaOF: ['D011']
RpT: ['D011']
RMTG: ['D013']
RTP: ['D013']
LA: ['D013', 'D014', 'D023']
LMH: ['D013', 'D016', 'D023']
LAC: ['D013']
LSTG: ['D014', 'D017']
LAF: ['D014', 'D016']
LF: ['D014', 'D030']
LMFG: ['D014']
LOF: ['D014', 'D015', 'D017', 'D023']
LSMA: ['D014', 'D030']
RHC: ['D015']
ROPR: ['D015']
RMF: ['D015']
RASF: ['D015']
LHC: ['D015']
LdAC: ['D016']
LOFAC: ['D016', 'D018']
LSF: ['D016']
LpCING: ['D016']
LEC: ['D016']
RSF: ['D016', 'D030']
LAI: ['D017']
LMI: ['D017']
LSI: ['D017']
LPI: ['D017']
LPHG: ['D017', 'D023']
ROFAC: ['D018', 'D023']
RSMA: ['D018', 'D030']
LCAUD: ['D018']
RPHG: ['D023']
RTO: ['D023']
RF: ['D

In [ ]:
#@title <font color = "\#8FBCFF">Channel count per patient
vv=0
for k,v in sorted(my_dict.items()):
    sv = sum(v.values())
    vv += sv
    print(f"{k} has {sv} bipolar channels")

D006 has 47 bipolar channels
D011 has 75 bipolar channels
D013 has 54 bipolar channels
D014 has 43 bipolar channels
D015 has 51 bipolar channels
D016 has 54 bipolar channels
D017 has 66 bipolar channels
D018 has 52 bipolar channels
D023 has 65 bipolar channels
D030 has 64 bipolar channels


In [ ]:
#@title <font color = "\#86FFFF">Total number of bipolar channels:
print(f'{vv}')

571
